In [5]:
print("Hello")

Hello


Import numpy and its concrete version

In [6]:
import concrete.numpy as cnp
import numpy as np

Formulate the logistic regression model that is given in the paper

In [7]:
#Weights and bias of the model
# age, systolic blood pressure, diastolic blood pressure, cholesterol level, height (inches), weight (pounds)
weights = np.array([0.072, 0.013, -0.029, 0.008, -0.053, 0.021])
bias = np.array([0])

def sigmoid(x):
    #e^x / (e^x+1)
    return 1 / (1 + np.exp(-x))

def L(data):
    return sigmoid(np.dot(data, weights) + bias)



Test the model with an example, we should obtain a probabilty between 0 and 1
(Examples taken from [LRM] of the paper: http://www.claudiaflowers.net/rsch8140/logistic_regression_example.htm)

In [8]:
#person1_data = np.array([64.0, 105.0, 68.0, 261.0, 66.0, 108.0])
person1_data = np.array([64, 105, 68, 261, 66, 108])
print(L(person1_data))

#person2_data = np.array([50.0, 160.0, 110.0, 261.0, 66.0, 145.0])
person2_data = np.array([50, 160, 110, 261, 66, 145])
print(L(person2_data))

[0.99230149]
[0.98409361]


Now try and do the whole thing encrypted with FHE
I follow the concrete-numpy documentation on compiling numpy functions
https://docs.zama.ai/concrete-numpy/stable/user/basics/compiling_and_executing.html?highlight=compile_numpy_function

In [9]:
#declare that the input variable x is encrypted (and of what shape?)
data_type = "encrypted"

#give an inputset of example inputs to determine the input bounds
#for us, inputs in the range [-3.755, 2.403] are allowed
#TODO (what is better, give min and max as only input examples or give several inputs inbetween?)
#TODO I think I need to give some range on the data input, i.e. age and so on, not the linear combination of data and weights
inputset = [np.array([64, 105, 68, 261, 66, 108]), np.array([50, 160, 110, 261, 66, 145])]
#inputset = [50, 261]

#compile the function to its homomorphic equivalent
compiler = cnp.NPFHECompiler(
    L, {"data": data_type},
)
circuit = compiler.compile_on_inputset(inputset)

RuntimeError: function you are trying to compile isn't supported for MLIR lowering

 %0 = 1                                # ClearScalar<uint1>
 %1 = 1                                # ClearScalar<uint1>
 %2 = 0                                # ClearScalar<uint1>
 %3 = [0]                              # ClearTensor<uint1, shape=(1,)>
 %4 = data                             # EncryptedTensor<uint9, shape=(6,)>
 %5 = [ 0.072  0 ... 53  0.021]        # ClearTensor<float64, shape=(6,)>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ only integer constants are supported
 %6 = dot(%4, %5)                      # EncryptedScalar<float64>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ only integer dot product is supported
 %7 = add(%6, %3)                      # EncryptedTensor<float64, shape=(1,)>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ only integer addition is supported
 %8 = sub(%2, %7)                      # E

!!! Big issue: floating point functions can be compiled but only univariate ones since that's what works with the lookup table
see Limitations of: https://docs.zama.ai/concrete-numpy/stable/user/tutorial/working_with_floating_points.html

In [10]:
print(str(compiler))

 %0 = 1                                # ClearScalar<uint1>
 %1 = 1                                # ClearScalar<uint1>
 %2 = 0                                # ClearScalar<uint1>
 %3 = [0]                              # ClearTensor<uint1, shape=(1,)>
 %4 = data                             # EncryptedTensor<uint9, shape=(6,)>
 %5 = [ 0.072  0 ... 53  0.021]        # ClearTensor<float64, shape=(6,)>
 %6 = dot(%4, %5)                      # EncryptedScalar<float64>
 %7 = add(%6, %3)                      # EncryptedTensor<float64, shape=(1,)>
 %8 = sub(%2, %7)                      # EncryptedTensor<float64, shape=(1,)>
 %9 = exp(%8)                          # EncryptedTensor<float64, shape=(1,)>
%10 = add(%9, %1)                      # EncryptedTensor<float64, shape=(1,)>
%11 = truediv(%0, %10)                 # EncryptedTensor<float64, shape=(1,)>
return %11


Now we try to do homomorphic evaluation

In [11]:
circuit.encrypt_run_decrypt(np.array([64, 105, 68, 261, 66, 108]))

NameError: name 'circuit' is not defined

Try and obtain the linear combination x via a separate function
This first attempt still doesn't work since we multiply by constants but they're floating points

In [20]:
def val_from_data(data):
    return np.dot(data, weights) + bias

compiler2 = cnp.NPFHECompiler(
    val_from_data, {"data": data_type},
)
circuit2 = compiler2.compile_on_inputset(inputset)

RuntimeError: function you are trying to compile isn't supported for MLIR lowering

%0 = [0]                              # ClearTensor<uint1, shape=(1,)>
%1 = data                             # EncryptedTensor<uint9, shape=(6,)>
%2 = [ 0.072  0 ... 53  0.021]        # ClearTensor<float64, shape=(6,)>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ only integer constants are supported
%3 = dot(%1, %2)                      # EncryptedScalar<float64>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ only integer dot product is supported
%4 = add(%3, %0)                      # EncryptedTensor<float64, shape=(1,)>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ only integer addition is supported
return %4

Next ideas:
input data should be integers so only problem is the weights, we can scale them and then reduce them again to obtain the single
floating point as input, then we should be able to evaluate the model homomorphically.

In [22]:
adjusted_weights = np.array([round(w * 1000) for w in weights])
print(adjusted_weights)

def val_from_data_adjusted(data):
    return np.dot(data, adjusted_weights) + bias

compiler3 = cnp.NPFHECompiler(
    val_from_data_adjusted, {"data": data_type},
)
circuit3 = compiler3.compile_on_inputset(inputset)

[ 72  13 -29   8 -53  21]


RuntimeError: max_bit_width of some nodes is too high for the current version of the compiler (maximum must be 8) which is not compatible with:

%0 = [0]                              # ClearTensor<uint1, shape=(1,)>
%1 = data                             # EncryptedTensor<uint9, shape=(6,)>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ 9 bits is not supported for the time being
%2 = [ 72  13 -29   8 -53  21]        # ClearTensor<int8, shape=(6,)>
%3 = dot(%1, %2)                      # EncryptedScalar<uint13>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ 13 bits is not supported for the time being
%4 = add(%3, %0)                      # EncryptedTensor<uint13, shape=(1,)>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ 13 bits is not supported for the time being
return %4

Latest error: bit_width of the input is not supported, I guess too large integers
I don't quite know why, the ints fit in a 128 wide range, i.e. 72+53 = 125 should not pose a problem
Maybe it is because it's an array or maybe we could shift the integers to actually be in 0 to 127, though that requires more pre- and post-computation, additionally to the scaling.